## Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

In [2]:
#training data
#inputs -> temp, humidity, rainfall
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 76, 90]], dtype='float32')

#targets -> apples, oranges
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [3]:
#inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

#### Model

In [4]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-1.5168,  0.9762, -0.5161],
        [-0.2573,  0.1951, -0.0146]], requires_grad=True)
tensor([0.6083, 0.4259], requires_grad=True)


In [5]:
#model
def model(x):
    return x @ w.t() + b        #@ -> matrix multiplication in pytorch; t-> transpose

In [6]:
#predictions
preds = model(inputs)
print(preds)

tensor([[ -66.9060,   -5.9202],
        [ -84.5467,   -6.7633],
        [ -30.4769,    3.3261],
        [-131.2260,  -17.9764],
        [ -76.3105,   -3.8227]], grad_fn=<AddBackward0>)


In [7]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
#loss function (mse)
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [9]:
loss = mse(preds, targets)
loss.sqrt()

tensor(131.4478, grad_fn=<SqrtBackward0>)

In [10]:
loss.backward()

In [11]:
print(w)
print(w.grad)

tensor([[-1.5168,  0.9762, -0.5161],
        [-0.2573,  0.1951, -0.0146]], requires_grad=True)
tensor([[-13008.5723, -12609.8057,  -9271.3828],
        [ -8142.5234,  -8728.9258,  -6154.1338]])


In [12]:
#resetting gradients back to 0 to prevent adding up
w.grad.zero_()
b.grad.zero_()
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


##### Gradient Descent

In [13]:
preds = model(inputs)
preds

tensor([[ -66.9060,   -5.9202],
        [ -84.5467,   -6.7633],
        [ -30.4769,    3.3261],
        [-131.2260,  -17.9764],
        [ -76.3105,   -3.8227]], grad_fn=<AddBackward0>)

In [14]:
loss = mse(preds, targets)
loss

tensor(17278.5332, grad_fn=<DivBackward0>)

In [15]:
#compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-13008.5723, -12609.8057,  -9271.3828],
        [ -8142.5234,  -8728.9258,  -6154.1338]])
tensor([-154.0932,  -98.2313])


In [16]:
#adjust weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [17]:
#new weights and biases
print(w)
print(b)

tensor([[-1.3867,  1.1023, -0.4234],
        [-0.1759,  0.2823,  0.0469]], requires_grad=True)
tensor([0.6098, 0.4269], requires_grad=True)


In [18]:
#re-calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(11857.4980, grad_fn=<DivBackward0>)

In [19]:
#training with 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [20]:
#new loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(183.1999, grad_fn=<DivBackward0>)


In [21]:
preds

tensor([[ 57.9453,  73.8378],
        [ 80.3832, 100.4609],
        [144.7362, 132.9337],
        [  6.4041,  52.7407],
        [ 82.4112, 103.6520]], grad_fn=<AddBackward0>)

In [22]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

##### Linear regression using PyTorch built-ins

In [23]:
import torch.nn as nn

In [24]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

##### Tensor Dataset and Dataloader

In [25]:
from torch.utils.data import TensorDataset

In [26]:
#define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [27]:
from torch.utils.data import DataLoader

In [28]:
#define dataloader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)


In [31]:
i=1
for xb, yb in train_dl:
    print(f'batch {i}:')
    print(xb)
    print(yb)
    i+=1

batch 1:
tensor([[103.,  43.,  36.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.]])
tensor([[ 20.,  38.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 57.,  69.],
        [ 82., 100.]])
batch 2:
tensor([[ 87., 135.,  57.],
        [ 69.,  96.,  70.],
        [ 68.,  97.,  70.],
        [101.,  44.,  37.],
        [ 91.,  88.,  64.]])
tensor([[118., 134.],
        [103., 119.],
        [102., 120.],
        [ 21.,  38.],
        [ 81., 101.]])
batch 3:
tensor([[ 87., 134.,  58.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 68.,  96.,  71.],
        [ 88., 134.,  59.]])
tensor([[119., 133.],
        [ 57.,  69.],
        [ 80., 102.],
        [104., 118.],
        [118., 132.]])


In [32]:
#do this instead of manually initializing weights and biases
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2620,  0.0181,  0.4393],
        [ 0.2135, -0.4776, -0.2041]], requires_grad=True)
Parameter containing:
tensor([-0.1046,  0.2470], requires_grad=True)


In [33]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2620,  0.0181,  0.4393],
         [ 0.2135, -0.4776, -0.2041]], requires_grad=True),
 Parameter containing:
 tensor([-0.1046,  0.2470], requires_grad=True)]

In [34]:
preds = model(inputs)
preds

tensor([[ 39.1221, -24.9438],
        [ 53.4430, -35.4171],
        [ 50.5902, -57.0177],
        [ 43.6504,  -6.0635],
        [ 50.4595, -45.1607],
        [ 39.3660, -24.2527],
        [ 53.8643, -35.1436],
        [ 51.2915, -57.0083],
        [ 43.4065,  -6.7546],
        [ 50.6369, -45.5783],
        [ 39.5433, -24.6703],
        [ 53.6869, -34.7260],
        [ 50.1690, -57.2912],
        [ 43.4731,  -5.6458],
        [ 50.2156, -45.8518]], grad_fn=<AddmmBackward0>)

In [35]:
#loss function
import torch.nn.functional as F

In [36]:
loss_fn = F.mse_loss

In [37]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(10137.9932, grad_fn=<MseLossBackward0>)
